In [112]:
import pandas as pd
import datetime
import plotly.graph_objects as go
from pathlib import Path

ModuleNotFoundError: No module named 'matplotlib'

In [55]:
loaded = {}

def load_ticker(ticker):
    if ticker in loaded:
        return loaded[ticker]
    filename = f"./csv/{ticker}.csv"
    print(f"Loading {filename}")
    df = pd.read_csv(filename, 
                       names=['Datetime', 'Open', 'High', 'Low', 'Close'], 
                       index_col='Datetime', 
                       parse_dates=True, 
                       infer_datetime_format=True)
    # Fix forexit timezone
    # See last post here: https://forextester.com/forum/viewtopic.php?t=3175
    # If I adjust the signal time to be one hour forward, I get the correct time.
    # So I need to adjust the forex time to be one hour backwards
    df = df.set_index(df.index.values - pd.Timedelta(hours=1))
    loaded[ticker] = df
    return df
                


In [97]:
class Signal:
    def __init__(self, ticker, datetime_str, entry_price, stop_loss, take_profit):
        # Signal time needs to be in the format '2020-02-01 19:34'
        self.datetime_str = datetime_str
        self.signal_time = datetime.datetime.strptime(datetime_str, '%Y-%m-%d %H:%M')
        self.entry_price = float(entry_price)
        self.stop_loss = float(stop_loss)
        self.take_profit = float(take_profit)
        self.type = "BUY" if take_profit > stop_loss else "SELL"
        self.ticker = ticker
        
    def __str__(self):
        return f"{self.ticker}: {self.datetime_str}: Entry: {self.entry_price}, SL: {self.stop_loss}, TP: {self.take_profit}"
        
    def set_stop_loss_pips(self, new_stop_loss):
        stop_loss_pair_val = new_stop_loss / pip_factor(self.ticker)
        if self.type == "BUY":
            self.stop_loss = self.entry_price - stop_loss_pair_val
        if self.type == "SELL":
            self.stop_loss = self.entry_price + stop_loss_pair_val

        
    def set_take_profit_pips(self, new_take_profit):
        take_profit_pair_val = new_take_profit / pip_factor(self.ticker)
        if self.type == "BUY":
            self.take_profit = self.entry_price + take_profit_pair_val
        if self.type == "SELL":
            self.take_profit = self.entry_price - take_profit_pair_val
        
        
def pip_factor(ticker):
    factor = 10000
    if "JPY" in ticker:
        factor = 100
        
    if "XAU" in ticker:
        factor = 10
    return factor

        
def analyse_signal(df, signal):
    real_entry = None
    for dt, ohlc in df.loc[signal.datetime_str:].iterrows():
        if not real_entry:
            real_entry = ohlc.Close # Assume it takes 1 min to enter the trade
            
        if signal.type == "BUY":
            if ohlc.High >= signal.take_profit:
                return abs(signal.take_profit - real_entry)

            if ohlc.Low <= signal.stop_loss:
                return -1 * abs(real_entry - signal.stop_loss)
            
        if signal.type == "SELL":
            if ohlc.Low <= signal.take_profit:
                return abs(signal.take_profit - real_entry)

            if ohlc.High >= signal.stop_loss:
                return -1 * abs(real_entry - signal.stop_loss)
        
    return 0.0
        

In [98]:
beta_tp1 = Signal("AUDCAD", "2020-01-14 13:19", 0.9020, 0.9075, 0.9000)
beta_tp2 = Signal("AUDCAD", "2020-01-14 13:19", 0.9020, 0.9075, 0.8970)
beta_tp3 = Signal("AUDCAD", "2020-01-14 13:19", 0.9020, 0.9075, 0.8920)
df = load_ticker("AUDCAD")
print("TP1", analyse_signal(df, beta_tp1) * pip_factor("AUDCAD"))
print("TP2", analyse_signal(df, beta_tp2) * pip_factor("AUDCAD"))
print("TP3", analyse_signal(df, beta_tp3) * pip_factor("AUDCAD"))



TP1 14.999999999999458
TP2 0.0
TP3 0.0


In [99]:
beta_tp1 = Signal("USDCAD", "2019-12-03 11:01", 1.33, 1.335, 1.328)
beta_tp2 = Signal("USDCAD", "2019-12-03 11:01", 1.33, 1.335, 1.325)
beta_tp3 = Signal("USDCAD", "2019-12-03 11:01", 1.33, 1.335, 1.32)

df = load_ticker("USDCAD")

print("TP1", analyse_signal(df, beta_tp1) * pip_factor("USDCAD"))
print("TP2", analyse_signal(df, beta_tp2) * pip_factor("USDCAD"))
print("TP3", analyse_signal(df, beta_tp3) * pip_factor("USDCAD"))

# Morning Richie
# Next steps:
# Extract all the Beta trades into a csv file
# Write something to read the csv and convert them to trades

# Answer a question:
# How do the pip profit values change for each TP when I modify the Stop Loss size, start with TP1



TP1 19.000000000000128
TP2 49.000000000001265
TP3 99.0000000000002


In [100]:
beta_tp1 = Signal("NZDCAD", "2019-12-04 12:44", 0.8656, 0.8715, 0.8636)
df = load_ticker("NZDCAD")
print("TP1", analyse_signal(df, beta_tp1) * pip_factor("NZDCAD"))


TP1 19.000000000000128


In [101]:
from collections import defaultdict

def load_beta_trades(filename):
    signals = defaultdict(list)
    with open(filename, 'r') as f:
        next(f) # Skip header row
        for signal in f:
            tokens = signal.split(",")
            datetime = tokens[0].strip()
            ticker = tokens[1].strip()
            signal_type = tokens[2].strip()
            entry = tokens[3].strip()
            tp1 = tokens[4].strip()
            tp2 = tokens[6].strip()
            tp3 = tokens[8].strip()
            sl = tokens[10].strip()
            signals["TP1"].append(Signal(ticker, datetime, entry, sl, tp1))
            signals["TP2"].append(Signal(ticker, datetime, entry, sl, tp2))
            signals["TP3"].append(Signal(ticker, datetime, entry, sl, tp3))
    return signals

In [109]:
# TP1 Analysis using trade signals as given

signals = load_beta_trades("./beta/2019-Nov+Dec-BETA.csv")
total_profit = 0.0
wins = 0
losses = 0
for signal in signals["TP1"]:
    df = load_ticker(signal.ticker)
    profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
    if profit > 0:
        wins += 1
    else:
        losses += 1
    total_profit += profit
    #print(signal, profit)
print("Default:", total_profit, wins, losses)

Default: 144.09999999999604 35 7


In [140]:
# TP1 Stop Loss analysis
stops = range(20, 51, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP1"]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation for TP1",
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


20 312.79999999999774 31 11
21 303.80000000000115 31 11
22 294.80000000000115 31 11
23 285.7999999999989 31 11
24 320.8000000000044 32 10
25 357.800000000002 33 9
26 350.7999999999968 33 9
27 343.80000000000143 33 9
28 336.7999999999995 33 9
29 329.7999999999999 33 9
30 322.8000000000023 33 9
31 315.799999999996 33 9
32 361.89999999999986 34 8
33 355.8999999999979 34 8
34 349.89999999999816 34 8
35 398.9000000000001 35 7
36 393.8999999999952 35 7
37 388.899999999996 35 7
38 383.89999999999736 35 7
39 378.89999999999816 35 7
40 373.8999999999967 35 7
41 368.89999999999804 35 7
42 363.89999999999884 35 7
43 358.89999999999736 35 7
44 353.90000000000094 35 7
45 348.89999999999947 35 7
46 343.8999999999975 35 7
47 338.8999999999983 35 7
48 333.8999999999968 35 7
49 328.90000000000043 35 7
50 323.89999999999895 35 7


In [133]:
# TP2 Stop loss analysis

stops = range(15, 35, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP2"]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    
go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation for TP2",
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


15 627.7999999999927 21 21
16 608.7999999999898 21 21
17 589.7999999999911 21 21
18 638.7999999999864 22 20
19 620.7999999999947 22 20
20 742.7999999999948 24 18
21 726.7999999999977 24 18
22 782.7999999999985 25 17
23 767.7999999999971 25 17
24 826.8000000000014 26 16
25 812.8000000000008 26 16
26 798.799999999993 26 16
27 784.7999999999971 26 16
28 770.7999999999961 26 16
29 756.7999999999976 26 16
30 742.7999999999992 26 16
31 728.7999999999896 26 16
32 714.7999999999967 26 16
33 700.7999999999928 26 16
34 686.799999999994 26 16


In [137]:
# TP3 Stop loss analysis

stops = range(20, 31, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation for TP3",
    xaxis_title="Take Profit (pips)",
    yaxis_title="Profit (pips)",
    title_x=0.5
)
fig.show()


20 763.7999999999939 14 28
21 737.8000000000045 14 28
22 818.8000000000008 15 27
23 793.7999999999954 15 27
24 1016.8000000000073 17 25
25 993.8000000000027 17 25
26 970.7999999999896 17 25
27 947.8000000000002 17 25
28 924.799999999996 17 25
29 901.7999999999938 17 25
30 878.8000000000021 17 25


In [161]:
# Take Profit analysis

stop_loss = 24
take_profits = range(65, 101, 1)
profits = []

for take_profit in take_profits:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop_loss)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(take_profit, total_profit, wins, losses)
    

65 1058.8000000000038 24 18
66 1082.8 24 18
67 1106.8000000000034 24 18
68 1130.8000000000075 24 18
69 1154.7999999999963 24 18
70 1178.8 24 18
71 1107.800000000008 23 19
72 1130.800000000011 23 19
73 1056.8000000000147 22 20
74 1078.8000000000054 22 20
75 1100.8000000000084 22 20
76 1022.8000000000173 21 21
77 1043.8000000000086 21 21
78 1064.800000000011 21 21
79 1085.800000000014 21 21
80 1002.800000000006 20 22
81 1022.8000000000083 20 22
82 1042.8000000000002 20 22
83 1062.800000000003 20 22
84 1082.800000000005 20 22
85 1102.799999999997 20 22
86 1122.800000000005 20 22
87 1142.800000000007 20 22
88 1162.7999999999993 20 22
89 1182.8000000000013 20 22
90 1088.8000000000036 19 23
91 1107.8000000000143 19 23
92 1126.8000000000156 19 23
93 1028.8000000000175 18 24
94 1046.8000000000113 18 24
95 1064.8000000000127 18 24
96 962.800000000013 17 25
97 979.8000000000137 17 25
98 996.8000000000144 17 25
99 1013.8000000000094 17 25
100 1030.80000000001 17 25


In [162]:
fig = go.Figure(data=go.Scatter(x=list(take_profits), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Take Profit Optimisation (Stop Loss = 24 pips)",
    title_x=0.5,
    xaxis_title="Take Profit Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [168]:
# Stop Loss analysis

take_profit = 70
stops = range(20, 30, 1)
profits = []

for stop in stops:
    total_profit = 0.0
    wins = 0
    losses = 0
    for signal in signals["TP3"]:
        signal.set_stop_loss_pips(stop)
        signal.set_take_profit_pips(take_profit)
        df = load_ticker(signal.ticker)
        profit = analyse_signal(df, signal) * pip_factor(signal.ticker)
        if profit > 0:
            wins += 1
        else:
            losses += 1
        total_profit += profit
        #print(signal, profit)
    profits.append(total_profit)
    print(stop, total_profit, wins, losses)
    

20 1062.799999999991 22 20
21 1044.7999999999956 22 20
22 1118.7999999999963 23 19
23 1101.7999999999943 23 19
24 1178.8 24 18
25 1162.799999999999 24 18
26 1146.7999999999902 24 18
27 1130.7999999999947 24 18
28 1114.7999999999934 24 18
29 1098.7999999999945 24 18


In [169]:
fig = go.Figure(data=go.Scatter(x=list(stops), y=profits))
fig.update_layout(
    title="BETA (Nov & Dec 2019 Signals) Stop Loss Optimisation (Take Profit = 59 pips)",
    title_x=0.5,
    xaxis_title="Stop Loss Level (pips)",
    yaxis_title="Profit (pips)"
)
fig.show()

In [ ]:
# Example plotly candlestick plot

dfp = df.loc["2019-01-14 21:00":"2019-01-14 22:00"]
fig = go.Figure(data=[go.Candlestick(x=dfp.index,
                open=dfp['Open'],
                high=dfp['High'],
                low=dfp['Low'],
                close=dfp['Close'])])

fig.show()